In [36]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [2]:
dataset = pd.read_csv("../data/pca/pca_fingerprint_results.csv").drop(["Unnamed: 0"], axis=1)
dataset

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,10.996596,-13.581953,-8.045430,-7.038271,-0.383731,-3.875451,-0.990448,1.297883,-0.574683,-1.370185,...,-0.424665,1.330598,-0.871246,1.463499,-0.420871,-0.112825,0.588864,-0.405024,1.157946,1.0
1,14.380604,-8.148405,-6.970214,-6.883370,-3.731843,2.294834,-2.794362,2.756995,1.141011,4.466174,...,0.104833,2.173534,0.188480,0.616560,1.007843,-0.985466,-1.090612,3.295126,1.582262,1.0
2,5.898082,-3.955692,-0.753455,0.755042,-5.912044,4.227601,-4.860235,4.010481,0.751071,6.718320,...,-1.402074,2.024921,-4.478798,2.390832,-4.388051,-0.940372,-5.093709,-0.226873,-3.592670,1.0
3,16.441638,-3.625168,-4.445587,-4.663273,-2.789747,1.248966,-2.006066,1.979731,-1.475145,0.365042,...,-1.109404,1.535672,-0.656872,-0.573160,-0.556370,-0.678389,0.096279,1.865521,0.709983,1.0
4,15.227068,-3.499011,-7.350787,-2.818675,-1.694236,-2.790198,-1.907673,1.458892,-2.542744,0.429509,...,-0.822749,0.278967,-0.629833,-0.808339,-0.787193,-1.012290,-0.291861,1.405601,0.632089,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7791,-6.519184,-1.975179,1.934837,1.355185,-1.104766,-1.073995,0.635247,-2.174292,0.351231,2.158437,...,0.629990,-0.609683,0.000688,0.777685,0.717557,1.148067,0.361464,-0.174682,-0.640264,2.0
7792,-8.936481,-3.827097,0.618619,0.926596,-0.369385,-0.559415,2.782636,-5.674792,6.605094,0.775494,...,-0.228395,0.183583,1.194140,-0.755381,-0.534693,0.027072,0.581102,-0.071217,0.163577,2.0
7793,-10.134237,-5.339714,-0.482419,-0.843599,-0.010870,0.205482,0.433432,-1.170274,1.914583,0.203546,...,-0.941713,0.270500,2.435655,-1.984606,-1.275028,-0.320185,0.439532,0.378725,0.637877,2.0
7794,-8.829808,-3.759077,0.602438,0.875080,-0.354298,-0.536431,2.552713,-5.185598,5.912753,0.736547,...,-0.726559,0.360763,2.146776,-1.507715,-0.910828,0.172310,0.735103,-0.028415,0.474641,2.0


In [3]:
dataset["target"].value_counts()

2.0    5118
0.0    2322
1.0      78
Name: target, dtype: int64

# 1. Split Data between Ligand and Decoy compounds

In [4]:
ligand_df = dataset[dataset["target"] == 1].reset_index(drop=True)
decoy_df = dataset[dataset["target"] == 0].reset_index(drop=True)

In [5]:
ligand_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,10.996596,-13.581953,-8.045430,-7.038271,-0.383731,-3.875451,-0.990448,1.297883,-0.574683,-1.370185,...,-0.424665,1.330598,-0.871246,1.463499,-0.420871,-0.112825,0.588864,-0.405024,1.157946,1.0
1,14.380604,-8.148405,-6.970214,-6.883370,-3.731843,2.294834,-2.794362,2.756995,1.141011,4.466174,...,0.104833,2.173534,0.188480,0.616560,1.007843,-0.985466,-1.090612,3.295126,1.582262,1.0
2,5.898082,-3.955692,-0.753455,0.755042,-5.912044,4.227601,-4.860235,4.010481,0.751071,6.718320,...,-1.402074,2.024921,-4.478798,2.390832,-4.388051,-0.940372,-5.093709,-0.226873,-3.592670,1.0
3,16.441638,-3.625168,-4.445587,-4.663273,-2.789747,1.248966,-2.006066,1.979731,-1.475145,0.365042,...,-1.109404,1.535672,-0.656872,-0.573160,-0.556370,-0.678389,0.096279,1.865521,0.709983,1.0
4,15.227068,-3.499011,-7.350787,-2.818675,-1.694236,-2.790198,-1.907673,1.458892,-2.542744,0.429509,...,-0.822749,0.278967,-0.629833,-0.808339,-0.787193,-1.012290,-0.291861,1.405601,0.632089,1.0
5,17.130025,-3.808420,-4.924050,-5.212040,-3.039766,1.531539,-2.384644,2.428395,-1.377488,1.411122,...,-0.760231,1.957892,-0.244268,0.139777,-0.602629,-1.239124,0.537542,2.054770,-0.283153,1.0
6,17.285471,-3.714313,-4.792839,-5.279551,-3.026864,1.427829,-2.336943,2.374500,-1.547378,1.280448,...,-0.687859,1.833940,-0.232216,-0.011662,-0.657755,-1.365048,0.406031,2.046955,-0.238376,1.0
7,4.140540,2.339490,0.647358,3.806197,-2.496182,-0.632273,-2.412149,1.656912,-4.781378,2.059584,...,-0.535720,0.924843,0.020154,-0.934424,-0.212801,-0.121974,-0.614628,-0.714652,-0.746142,1.0
8,12.130178,-7.446414,-7.820945,-3.335555,-2.635230,-2.790517,-2.688600,1.825950,-0.187694,1.658327,...,-0.147585,0.812843,-0.513583,-0.009741,0.147273,-0.222888,-0.450167,0.563955,1.041704,1.0
9,19.761315,-6.560360,-11.011217,-8.690976,-4.372912,5.369039,-2.634858,4.442513,0.517369,5.069157,...,-0.127951,0.561762,0.236171,2.829602,-0.185255,0.670708,2.822490,-1.081907,1.582238,1.0


In [6]:
decoy_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,4.744462,-5.977049,-0.426462,6.875258,-4.944770,-0.081321,-3.388374,0.927046,0.149188,3.490263,...,-0.841939,-0.227289,-0.600790,0.881211,-0.815057,-0.257304,0.015659,-0.674682,0.787644,0.0
1,18.616656,-4.090988,-3.886856,-4.974335,3.391845,-3.123838,4.098292,-3.849454,-1.677661,2.203731,...,0.204354,-0.824051,0.509214,-0.993434,-0.302164,0.320085,-0.116181,0.826341,0.842240,0.0
2,-0.426720,-3.413793,-0.179881,2.605793,-1.900717,0.067323,-1.095765,-0.326148,2.884739,2.881889,...,1.641553,0.926994,0.199114,0.115075,-0.787457,0.448487,0.411019,1.884834,-0.346280,0.0
3,14.678645,-8.750820,-7.167093,-7.935760,3.052605,-3.229477,6.865956,-7.634898,3.610064,6.255011,...,0.083290,-1.207004,1.077431,1.332146,1.404884,-0.400601,0.884124,-0.884929,1.484342,0.0
4,9.593584,-8.638656,3.051121,-2.899846,-5.661458,11.337477,-1.534611,1.312930,3.814845,3.532463,...,-0.659130,-0.137506,-0.142194,2.452162,-1.227538,1.445466,0.590282,0.174918,1.519953,0.0
5,18.728722,-5.408195,-0.869281,-8.146121,1.273378,1.121468,4.051868,-4.055572,1.517462,5.561896,...,1.877143,0.393682,-0.266504,-3.336437,1.344660,0.728245,-1.265665,-1.495014,-0.315329,0.0
6,17.439171,1.083316,-3.187567,-5.831439,2.344051,-3.442174,1.808126,-2.052169,-0.694774,3.994282,...,-0.316428,1.091043,0.358581,0.363825,-0.123477,-0.087290,-1.313585,-0.338700,0.991103,0.0
7,19.384751,-3.886428,-0.913879,-7.904117,0.476356,-0.145533,2.025449,-2.146869,-0.776525,-4.475033,...,1.156903,0.530610,0.095308,-1.611730,0.646496,-0.052421,-0.001889,-0.719275,-2.319058,0.0
8,10.429442,-2.717079,6.280579,12.256169,11.013629,-1.207916,-10.373586,3.687788,-5.033247,-3.989095,...,-0.199597,-0.761466,0.487847,-1.508008,-0.032794,-0.702647,-0.542907,2.001569,-0.279164,0.0
9,1.684616,-5.248740,-0.357764,4.604446,-4.027288,0.880609,-2.503621,-0.071119,2.089897,4.572006,...,0.085649,-0.641270,-1.292863,-0.793326,-0.512335,0.338810,-0.464532,0.654238,0.639348,0.0


In [8]:
ligand_df.info() # have 78 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Columns: 196 entries, 0 to target
dtypes: float64(196)
memory usage: 119.6 KB


In [9]:
decoy_df.info() # have 2322 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Columns: 196 entries, 0 to target
dtypes: float64(196)
memory usage: 3.5 MB


# 2. Take 150 random samples of decoy compounds

In [13]:
decoy_samples = decoy_df.sample(n=150, random_state=42).reset_index(drop=True) # Take 150 random samples of decoy compound
ligand_samples = ligand_df.copy() # Take all 78 samples of ligand compound

In [14]:
decoy_samples

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,18.880061,-4.994294,8.530817,-9.332611,-1.937971,14.112943,3.051646,-0.729857,-2.752432,-4.623634,...,-0.513131,0.356183,-1.425115,1.180736,-0.025718,0.003734,-0.069990,-0.781597,-0.534617,0.0
1,18.018535,-4.411982,-5.501023,-0.608663,2.318831,-4.244598,5.012211,-6.809540,-1.524864,-6.665108,...,1.011021,-0.212266,1.359700,-1.608726,-0.242972,0.879927,0.102192,0.059700,-0.257719,0.0
2,5.753552,4.630377,2.118046,1.371330,-2.849891,-3.450770,-2.473278,2.407347,-2.680034,-1.232313,...,3.402360,-1.290986,-0.575087,0.910114,0.878836,0.414832,1.653018,-1.568800,0.057982,0.0
3,19.625465,-3.919013,-3.301671,-4.897107,1.066236,-3.324209,0.640875,1.344652,-2.727897,3.308628,...,-1.582160,0.320474,0.466842,-0.638386,-0.898458,-0.555871,-0.888306,-2.356396,-2.490211,0.0
4,8.347836,-11.055225,-7.625973,2.714140,-3.301295,-1.167390,-1.087602,0.153429,1.417243,-3.736924,...,-0.116379,-0.160098,-0.664440,-0.087544,-0.183778,0.224221,-0.355693,0.946407,0.679942,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,6.084648,-5.344435,5.793573,1.273343,-6.355497,6.207341,-4.041189,3.617409,-2.512268,3.816984,...,1.596883,0.184629,-1.516802,-0.025716,0.007453,-1.216665,0.958340,0.131660,1.421456,0.0
146,14.731134,-0.282547,2.316027,-1.277033,0.450687,-2.674330,0.141120,1.864866,-1.693391,5.536296,...,-1.530411,-1.180775,-1.807555,2.608016,0.256664,1.507725,-1.961589,-0.628435,-2.067867,0.0
147,4.052502,-4.837786,9.852422,15.134935,0.663602,-8.219100,16.499916,19.007456,6.784903,1.285153,...,-0.143587,-1.891934,2.800223,3.544683,4.569308,-0.799104,-1.068198,-5.434485,-0.706954,0.0
148,3.610731,11.844198,0.284205,0.791929,-2.792724,-2.385253,-2.179349,0.383515,1.027705,1.818874,...,-0.368263,0.158645,0.167714,-0.323520,-1.404706,0.648104,0.322210,1.581405,1.032171,0.0


In [15]:
ligand_samples

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,10.996596,-13.581953,-8.045430,-7.038271,-0.383731,-3.875451,-0.990448,1.297883,-0.574683,-1.370185,...,-0.424665,1.330598,-0.871246,1.463499,-0.420871,-0.112825,0.588864,-0.405024,1.157946,1.0
1,14.380604,-8.148405,-6.970214,-6.883370,-3.731843,2.294834,-2.794362,2.756995,1.141011,4.466174,...,0.104833,2.173534,0.188480,0.616560,1.007843,-0.985466,-1.090612,3.295126,1.582262,1.0
2,5.898082,-3.955692,-0.753455,0.755042,-5.912044,4.227601,-4.860235,4.010481,0.751071,6.718320,...,-1.402074,2.024921,-4.478798,2.390832,-4.388051,-0.940372,-5.093709,-0.226873,-3.592670,1.0
3,16.441638,-3.625168,-4.445587,-4.663273,-2.789747,1.248966,-2.006066,1.979731,-1.475145,0.365042,...,-1.109404,1.535672,-0.656872,-0.573160,-0.556370,-0.678389,0.096279,1.865521,0.709983,1.0
4,15.227068,-3.499011,-7.350787,-2.818675,-1.694236,-2.790198,-1.907673,1.458892,-2.542744,0.429509,...,-0.822749,0.278967,-0.629833,-0.808339,-0.787193,-1.012290,-0.291861,1.405601,0.632089,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,14.066839,-6.491627,-2.091173,-2.528079,-2.298534,-4.824863,-2.297510,0.239755,2.397044,0.689745,...,-0.294248,-0.749754,-1.624025,0.233379,1.680851,0.573061,-0.575019,0.781579,-2.756030,1.0
74,15.599291,6.118581,-0.615089,-9.278798,-1.751151,-3.066809,0.152417,-1.444177,4.582531,-2.683279,...,-1.204551,0.991233,0.629331,0.908100,0.701706,0.135488,-0.173850,-0.300493,0.348620,1.0
75,5.874727,14.787542,6.059764,-2.985573,-6.662480,-1.436483,-4.851312,2.747031,0.743673,0.513223,...,0.576287,0.130856,0.746882,-1.025057,-0.366139,0.195104,-0.208159,-0.252302,0.716349,1.0
76,9.107021,-1.187344,-0.628291,6.129179,-2.053594,-0.193606,-2.227678,2.275277,-7.771191,0.467247,...,0.669240,-0.410826,-1.060554,-0.011827,0.791731,0.072175,-0.240811,1.110101,-1.165583,1.0


# 3. Concatenate data of randomly sampled Decoy with Ligand

In [17]:
sampled_data = pd.concat([decoy_samples, ligand_samples], axis=0).reset_index(drop=True)
sampled_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,18.880061,-4.994294,8.530817,-9.332611,-1.937971,14.112943,3.051646,-0.729857,-2.752432,-4.623634,...,-0.513131,0.356183,-1.425115,1.180736,-0.025718,0.003734,-0.069990,-0.781597,-0.534617,0.0
1,18.018535,-4.411982,-5.501023,-0.608663,2.318831,-4.244598,5.012211,-6.809540,-1.524864,-6.665108,...,1.011021,-0.212266,1.359700,-1.608726,-0.242972,0.879927,0.102192,0.059700,-0.257719,0.0
2,5.753552,4.630377,2.118046,1.371330,-2.849891,-3.450770,-2.473278,2.407347,-2.680034,-1.232313,...,3.402360,-1.290986,-0.575087,0.910114,0.878836,0.414832,1.653018,-1.568800,0.057982,0.0
3,19.625465,-3.919013,-3.301671,-4.897107,1.066236,-3.324209,0.640875,1.344652,-2.727897,3.308628,...,-1.582160,0.320474,0.466842,-0.638386,-0.898458,-0.555871,-0.888306,-2.356396,-2.490211,0.0
4,8.347836,-11.055225,-7.625973,2.714140,-3.301295,-1.167390,-1.087602,0.153429,1.417243,-3.736924,...,-0.116379,-0.160098,-0.664440,-0.087544,-0.183778,0.224221,-0.355693,0.946407,0.679942,0.0
5,13.200631,-6.584079,-6.912367,-3.610110,-1.902176,-2.730864,-1.271967,0.254899,-0.699497,2.545727,...,-0.211441,0.934955,-0.026601,0.493040,0.014791,0.289944,0.585126,-1.288744,-0.146990,0.0
6,12.173522,-5.750077,-3.122149,-5.620696,-1.487291,-3.899745,-1.959703,1.055523,-0.840163,2.479718,...,-0.488258,-2.693688,1.233251,1.077838,-0.684457,0.743598,0.102782,-1.645835,-2.029369,0.0
7,5.310317,-6.276273,3.820207,9.111078,6.034002,-1.061456,-9.672827,1.295371,0.225923,1.997158,...,0.442313,0.501329,0.426498,-0.488529,-0.524402,1.148536,0.110466,-1.197591,0.560126,0.0
8,6.808834,14.323019,2.691168,-1.621573,-6.166772,-5.632995,-5.608810,2.962218,-0.192471,1.182005,...,1.061482,-0.419710,0.444109,-0.771008,-1.074159,-0.180094,0.204536,-0.631238,1.074507,0.0
9,13.433529,-9.983830,-5.626570,-2.858065,-1.743954,-2.735526,-1.372519,0.177090,-0.252239,0.143873,...,0.217436,-0.081115,1.003618,-1.141962,0.633082,-0.323377,0.134449,-0.687858,0.298180,0.0


In [21]:
sampled_data.info() # Have 228 entries, 150 of decoys and 78 of ligands

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Columns: 196 entries, 0 to target
dtypes: float64(196)
memory usage: 349.3 KB


# 4. Oversampling of Ligand using SMOTE

In [26]:
# Separate features and labels
X = sampled_data.drop(columns=['target'])
y = sampled_data['target']

In [27]:
# Initiate SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=7)

# Do oversampling
X_resampled, y_resampled = smote.fit_resample(X, y)

In [28]:
# Create a new DataFrame with the oversampled data
oversampled_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='target')], axis=1)
oversampled_data

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,18.880061,-4.994294,8.530817,-9.332611,-1.937971,14.112943,3.051646,-0.729857,-2.752432,-4.623634,...,-0.513131,0.356183,-1.425115,1.180736,-0.025718,0.003734,-0.069990,-0.781597,-0.534617,0.0
1,18.018535,-4.411982,-5.501023,-0.608663,2.318831,-4.244598,5.012211,-6.809540,-1.524864,-6.665108,...,1.011021,-0.212266,1.359700,-1.608726,-0.242972,0.879927,0.102192,0.059700,-0.257719,0.0
2,5.753552,4.630377,2.118046,1.371330,-2.849891,-3.450770,-2.473278,2.407347,-2.680034,-1.232313,...,3.402360,-1.290986,-0.575087,0.910114,0.878836,0.414832,1.653018,-1.568800,0.057982,0.0
3,19.625465,-3.919013,-3.301671,-4.897107,1.066236,-3.324209,0.640875,1.344652,-2.727897,3.308628,...,-1.582160,0.320474,0.466842,-0.638386,-0.898458,-0.555871,-0.888306,-2.356396,-2.490211,0.0
4,8.347836,-11.055225,-7.625973,2.714140,-3.301295,-1.167390,-1.087602,0.153429,1.417243,-3.736924,...,-0.116379,-0.160098,-0.664440,-0.087544,-0.183778,0.224221,-0.355693,0.946407,0.679942,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,5.985160,12.779421,3.230932,-1.558969,-6.045938,0.241865,-5.473675,4.361625,-0.249197,1.695995,...,-0.915841,0.476160,-0.158109,-0.985972,-0.777564,-0.300246,0.151411,-0.393583,-0.297312,1.0
296,12.871567,-5.836833,-8.493318,6.386544,-3.031352,1.297841,-2.587385,2.111745,-3.450812,-1.700955,...,-0.012749,0.617378,-0.853879,2.227534,1.025873,-0.638069,-0.183974,-0.059744,-0.861809,1.0
297,5.724934,13.276735,-0.890055,-2.016508,-7.918346,-5.399766,-10.249366,7.243133,1.054543,2.582620,...,-0.064411,-0.641454,-0.412888,0.643862,-1.093087,0.250658,0.393813,-0.754105,-1.153816,1.0
298,16.751409,-4.134484,-5.920277,-2.643875,2.578754,-3.481571,-1.590502,-1.337279,-1.119437,-1.102343,...,-0.149072,0.616654,-0.908813,-1.023145,-0.441327,-1.077154,-0.666029,1.207927,-0.274054,1.0


In [29]:
oversampled_data.info() # have 300 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Columns: 196 entries, 0 to target
dtypes: float64(196)
memory usage: 459.5 KB


In [31]:
oversampled_data["target"].value_counts() # Balanced data between ligand and decoy

0.0    150
1.0    150
Name: target, dtype: int64

In [34]:
oversampled_data.to_csv("../data/resampled/oversampled_data.csv")

# 5. Undersampling of Decoy using RandomRandomSampler

In [37]:
# Initiate undersampler
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=3)

# Do undersamping
X_resampled, y_resampled = undersampler.fit_resample(X, y)

In [39]:
# Create a new DataFrame with the undersampled data
undersampled_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='target')], axis=1).reset_index(drop=True)
undersampled_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,target
0,9.520672,-10.426676,-6.104782,2.647147,-3.113361,-1.877094,-2.359998,2.278818,1.600676,-2.396448,...,1.229301,0.069955,-0.835422,0.122478,0.064584,1.775256,-1.546339,-0.659141,0.895978,0.0
1,19.625465,-3.919013,-3.301671,-4.897107,1.066236,-3.324209,0.640875,1.344652,-2.727897,3.308628,...,-1.582160,0.320474,0.466842,-0.638386,-0.898458,-0.555871,-0.888306,-2.356396,-2.490211,0.0
2,9.741854,0.543330,3.887575,7.243629,0.099781,-1.441249,1.985987,-1.846728,-2.067276,-3.399294,...,-1.086686,-0.868464,-0.600756,0.376293,0.619421,-0.625389,0.931297,-0.041456,-0.862373,0.0
3,16.943916,-5.327857,1.698061,-3.479702,-1.624494,8.763123,2.252987,-0.744259,-4.383421,-5.198033,...,2.223409,-0.040657,-0.743231,0.219092,1.459336,1.317327,-1.052049,0.329592,1.144037,0.0
4,3.753543,-4.444411,-2.525337,10.336373,-3.509968,3.957016,-5.926953,7.870994,3.934815,-0.834145,...,-2.071532,0.940542,-1.408797,0.921555,-0.631350,1.606957,-1.158639,0.486346,-0.320635,0.0
5,15.727161,-3.247678,-3.697253,-4.090843,1.179086,-3.597200,1.056444,-1.827016,-2.820173,-0.460452,...,-0.874969,-1.029531,-0.840625,-0.253573,-0.679838,-0.152583,0.320905,0.746303,-1.150747,0.0
6,7.160728,13.684540,0.694001,-3.116596,-7.721842,-6.986336,-9.246636,6.028604,0.535942,1.323493,...,-0.524937,-0.883307,-1.038432,-0.238390,-1.269999,1.037849,1.764635,-1.160518,-0.118282,0.0
7,12.173522,-5.750077,-3.122149,-5.620696,-1.487291,-3.899745,-1.959703,1.055523,-0.840163,2.479718,...,-0.488258,-2.693688,1.233251,1.077838,-0.684457,0.743598,0.102782,-1.645835,-2.029369,0.0
8,6.064972,2.124089,3.917095,2.228577,-3.092502,2.720841,-1.642097,1.137401,-4.626911,2.303526,...,0.036856,1.642571,0.275571,-0.747554,-0.019178,0.050068,0.124153,-1.090742,-0.749581,0.0
9,9.835565,-6.322459,2.606269,-0.165837,-3.785739,5.376883,-0.100222,1.150383,-1.411530,5.621892,...,-2.996782,2.957296,0.343780,0.780498,0.151058,1.547948,-1.129203,-0.040863,-0.515431,0.0


In [40]:
undersampled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Columns: 196 entries, 0 to target
dtypes: float64(196)
memory usage: 239.0 KB


In [41]:
undersampled_data["target"].value_counts() # Balanced data between ligand and decoy

0.0    78
1.0    78
Name: target, dtype: int64

In [42]:
undersampled_data.to_csv("../data/resampled/undersampled_data.csv")